<H2>View all basic combination definitions</H2>

In [ ]:
import pandas as pd
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import numpy as np
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.22.0")
if not lusas.existsDatabase():
    Exception("A model must be open before running this code")

db = lusas.database() # Reference to the database for convenience

# Directory to export to
import os
export_dir = os.path.expanduser("~/Desktop")
file_path = rf"{export_dir}\{db.getDBBasename()}-Combination Definitions.xlsx"
writer = pd.ExcelWriter(file_path)

# When calling LUSAS externally significant speed up is gained by disabling the UI
# The UI must be re-enabled otherwise it will appear locked to the user
if lusas.getMajorVersionNumber() >= 22 :lusas.enableUI(False)

Basic Combinations

In [ ]:
# Get all the basic combinations
all_basic_combinations = lusas.db().getLoadsets("Basic Combinations")

loadset_dict = {}
for loadset in db.getLoadsets():
    loadset_dict[loadset.getIDAndName()] = np.full(len(all_basic_combinations), np.nan)

# Create a list to store all the combination data
combs_data = []

for ic, combination in enumerate(all_basic_combinations):
    # Combination Name
    combs_data.append(combination.getIDAndNameAndDescription())    
    # Get a list of all the loadsets in the ocmbination
    ids = combination.getLoadcaseIDs()
    # Get a list of all the corresponding factors
    factors = combination.getFactors()
    # Append each loadset name followed by the factor used
    for i, id in enumerate(ids):
        loadset = db.getLoadset(id)
        loadset_dict[loadset.getIDAndName()][ic] = factors[i]

# Create a list for the column headers
column_headers = ["Combination"] 
# Use a pandas dataframe to export to xlsx format
df = pd.DataFrame(combs_data, columns= column_headers)

for load in loadset_dict.keys():
    factors = loadset_dict[load]
    if (~np.isnan(factors)).any():
        df[load] = factors

df.to_excel(writer, sheet_name="Basic Combinations")
# Print the top few elements of the dataframe as a check
print(df.head())

Smart Combinations

In [ ]:
# Get all the smart combinations
all_smart_combinations = lusas.db().getLoadsets("Smart Combinations")

loadset_dict = {}
for loadset in db.getLoadsets():
    loadset_dict[loadset.getIDAndName() + " [INF]"] = np.full(len(all_smart_combinations), np.nan)
    loadset_dict[loadset.getIDAndName() + " [SUP]"] = np.full(len(all_smart_combinations), np.nan)

# Create a list to store all the combination data
combs_data = []

for ic, combination in enumerate(all_smart_combinations):
    # Combination Name
    combs_data.append(combination.getIDAndNameAndDescription())    
    # Get a list of all the loadsets in the combination
    ids = combination.getLoadcaseIDs()
    # Get a list of all the corresponding permanent factors
    permanent_factors = combination.getPermanentFactors()
    # Get a list of all the corresponding variable factors
    variable_factors = combination.getVariableFactors()
    # Append each loadset name followed by the factor used
    for i, id in enumerate(ids):
        loadset = lusas.db().getLoadset(id).getIDAndName()
        # Note that the adverse factor is made up of a permanent and variable parts
        loadset_dict[loadset + " [INF]"][ic] = permanent_factors[i] 
        loadset_dict[loadset + " [SUP]"][ic] = permanent_factors[i]+variable_factors[i]

# Create a list for the column headers
column_headers = ["Combination"] 
# Use a pandas dataframe to export to xlsx format
df = pd.DataFrame(combs_data, columns= column_headers)

for load in loadset_dict.keys():
    factors = loadset_dict[load]
    if (~np.isnan(factors)).any():
        df[load] = factors

df.to_excel(writer, sheet_name="Smart Combinations")
# Print the top few elements of the dataframe as a check
print(df.head())

Envelopes

In [ ]:
# Get all the smart combinations
all_envelopes = lusas.db().getLoadsets("Envelopes")

loadset_dict = {}
for loadset in db.getLoadsets():
    loadset_dict[loadset.getIDAndName()] = np.full(len(all_envelopes), np.nan)

# Create a list to store all the combination data
combs_data = []

for ic, envelope in enumerate(all_envelopes):
    # Combination Name
    combs_data.append(envelope.getIDAndNameAndDescription())    
    # Get a list of all the loadsets in the combination
    ids = envelope.getLoadcaseIDs()
    for i, id in enumerate(ids):
        loadset = lusas.db().getLoadset(id)
        loadset_dict[loadset.getIDAndName()][ic] = 1

# Create a list for the column headers
column_headers = ["Envelope"] 
# Use a pandas dataframe to export to xlsx format
df = pd.DataFrame(combs_data, columns= column_headers)

for load in loadset_dict.keys():
    factors = loadset_dict[load]
    if (~np.isnan(factors)).any():
        df[load] = factors

df.to_excel(writer, sheet_name="Envelopes")
# Print the top few elements of the dataframe as a check
print(df.head())

In [ ]:
# Must close the writer to free up the file handle
writer.close()
# UI must be re-enabled
lusas.enableUI(True)